In [1]:
# import necesary packages:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import count



In [2]:
# We create a spark Session
spark = SparkSession.builder.appName("Jupyter Notebook").getOrCreate()

spark

In [3]:
# We load the data:
data = spark.read.json("movie.json")

In [4]:
# as we can see, each row does not represent a signle movie. So we will have to take this into account!
filtered_df = data.select('title').filter(data["title"] == 'Vengeance')
filtered_df.show()

+---------+
|    title|
+---------+
|Vengeance|
|Vengeance|
|Vengeance|
|Vengeance|
|Vengeance|
|Vengeance|
|Vengeance|
|Vengeance|
|Vengeance|
+---------+



In [5]:
# We have 56769 unique movies in our data set.
distinct_count = data.agg(countDistinct("title"))
distinct_count.show()

+------------+
|count(title)|
+------------+
|       56769|
+------------+



In [6]:
# As we said before we have more than one time the same movie in the dataset.
# So, we first keep only the genre and title and then we count distinct
filtered_data = data.select("genre",'title').filter(array_contains(data["genre"], "Comedy"))
filtered_data.agg(countDistinct("title",'genre')).show()

# We have 19,213 movies that are Comedy type
# These movies can also be of another type
# For example we counted also movies that are Comedy, Romance.


+-------------------+
|count(title, genre)|
+-------------------+
|              19213|
+-------------------+



In [7]:
# We find the statistics for the users_rating column:
statistics = data.select("users_rating").summary()

# We find the following:
# we have 62,056 user ratings, as many as the total number of movies in our data base
# The mean is 6, with a standar deviation of 1.3
# the min rating of a movie is 1 and the max rating of a movie is 9.9
# We also find informations about the quantiles
statistics.show()

+-------+-----------------+
|summary|     users_rating|
+-------+-----------------+
|  count|            62056|
|   mean|5.814105001933739|
| stddev|1.352186410172222|
|    min|              1.0|
|    25%|              5.0|
|    50%|              6.0|
|    75%|              6.7|
|    max|              9.9|
+-------+-----------------+



In [8]:
# We group by the rating column and count the number of occurances for each rating type:
grouped_data = data.groupBy("rating").count()

# We print the results:
grouped_data.show()

+---------+-----+
|   rating|count|
+---------+-----+
|Not Rated| 8080|
|       UA|    7|
|  Unrated| 1295|
|     TV-Y|   16|
| Approved| 6419|
|   Passed| 4488|
|     M/PG|   27|
|     null|20669|
|       PG| 3286|
|    TV-MA|  639|
|       18|    1|
|        M|   41|
|    TV-PG|  268|
|        C|    3|
|    NC-17|   22|
|        X|  152|
|        R|11368|
|        G|  801|
|    TV-14|  452|
|       GP|  105|
+---------+-----+
only showing top 20 rows

